In [1]:
import json
import pandas as pd
import re
import tensorflow as tf
from glove import Corpus, Glove
from gensim.models.word2vec import Text8Corpus


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\ProgramData\Anaconda3\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df = pd.read_json("dataset/train-v1.1.json", encoding="utf-8")

In [3]:
df.head()

,data,version
0,"{'title': 'University_of_Notre_Dame', 'paragra...",1.1
1,"{'title': 'Beyoncé', 'paragraphs': [{'context'...",1.1
2,"{'title': 'Montana', 'paragraphs': [{'context'...",1.1
3,"{'title': 'Genocide', 'paragraphs': [{'context...",1.1
4,"{'title': 'Antibiotics', 'paragraphs': [{'cont...",1.1


In [12]:
df[0:1]['data']

0    {'title': 'University_of_Notre_Dame', 'paragra...
Name: data, dtype: object

In [13]:
df.iloc[5]['data']['paragraphs'][1]

{'context': 'At the age of 21 he settled in Paris. Thereafter, during the last 18 years of his life, he gave only some 30 public performances, preferring the more intimate atmosphere of the salon. He supported himself by selling his compositions and teaching piano, for which he was in high demand. Chopin formed a friendship with Franz Liszt and was admired by many of his musical contemporaries, including Robert Schumann. In 1835 he obtained French citizenship. After a failed engagement to Maria Wodzińska, from 1837 to 1847 he maintained an often troubled relationship with the French writer George Sand. A brief and unhappy visit to Majorca with Sand in 1838–39 was one of his most productive periods of composition. In his last years, he was financially supported by his admirer Jane Stirling, who also arranged for him to visit Scotland in 1848. Through most of his life, Chopin suffered from poor health. He died in Paris in 1849, probably of tuberculosis.',
 'qas': [{'answers': [{'answer_s

In [19]:
words = []
sentences = []
count = 0
for index, row in df.iterrows():    
    for each_para_list in row['data']['paragraphs']:
        sentList = each_para_list['context'].strip().split(". ")
        for sent in sentList:
            words.extend(sent.split())
            sentences.append(sent.split())
        for each_qas in each_para_list['qas']:
            words.extend(each_qas['question'].strip().split())
    
    if count > 2:
        break
    count += 1
#             sentences = sentences + each_qas['question']
    [w for w in words if w]
print(sentences)

[['Architecturally,', 'the', 'school', 'has', 'a', 'Catholic', 'character'], ['Atop', 'the', 'Main', "Building's", 'gold', 'dome', 'is', 'a', 'golden', 'statue', 'of', 'the', 'Virgin', 'Mary'], ['Immediately', 'in', 'front', 'of', 'the', 'Main', 'Building', 'and', 'facing', 'it,', 'is', 'a', 'copper', 'statue', 'of', 'Christ', 'with', 'arms', 'upraised', 'with', 'the', 'legend', '"Venite', 'Ad', 'Me', 'Omnes"'], ['Next', 'to', 'the', 'Main', 'Building', 'is', 'the', 'Basilica', 'of', 'the', 'Sacred', 'Heart'], ['Immediately', 'behind', 'the', 'basilica', 'is', 'the', 'Grotto,', 'a', 'Marian', 'place', 'of', 'prayer', 'and', 'reflection'], ['It', 'is', 'a', 'replica', 'of', 'the', 'grotto', 'at', 'Lourdes,', 'France', 'where', 'the', 'Virgin', 'Mary', 'reputedly', 'appeared', 'to', 'Saint', 'Bernadette', 'Soubirous', 'in', '1858'], ['At', 'the', 'end', 'of', 'the', 'main', 'drive', '(and', 'in', 'a', 'direct', 'line', 'that', 'connects', 'through', '3', 'statues', 'and', 'the', 'Gold', 

In [ ]:
vocab = {}
for w in words:
    if w in vocab:
        vocab[w] += 1
    else:
        vocab[w] = 1

In [ ]:
vocab_list = sorted(vocab, key=vocab.get, reverse=True)

In [7]:
corpus = Corpus()
# v = []
# v.append(sentences)
corpus.fit(sentences)

glove = Glove(no_components=100, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
# print(sentences[0])

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2


KeyboardInterrupt: 

In [8]:
glove.add_dictionary(corpus.dictionary)

In [17]:
glove.most_similar('friendship')

[('House', 0.987350995394992),
 ('approval', 0.9869599231870789),
 ('origin', 0.9867473641994609),
 ('90%', 0.9867324147209517)]

In [ ]:
import itertools
# p = r'C:\\Users\\Pratik\\Downloads\\text8'
sentencesTest = list(itertools.islice(Text8Corpus("dataset/text8"),None))

In [ ]:
sentencesTest[0]

In [ ]:
# Referred : https://towardsdatascience.com/lstm-by-example-using-tensorflow-feb0c1968537

def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 1-layer LSTM with n_hidden units.
    rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

vocab_size = len(dictionary)
n_input = 3
# number of units in RNN cell
n_hidden = 512
# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)